In [ ]:
# %%capture
!pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2 langchain_community ddgs

# Agents


In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate


# Load OpenAI's LLMs with LangChain
os.environ["OPENAI_API_KEY"] = ""
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
# Create the ReAct template
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do. When you use the Calculator tool, make sure to provide a valid numerical expression.
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [ ]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# Prepare tools
tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent
agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [ ]:
# What is the Price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)



> Entering new AgentExecutor chain...
I should use the duckduck tool to search for the current price of a MacBook Pro in USD and then use the Calculator tool to convert it to EUR.
Action: duckduck
Action Input: "current price of MacBook Pro in USD"snippet: Discover the MacBook Pro laptop with the M4 family of chips, built for Apple Intelligence. Get credit when you trade in an eligible Mac . Buy now.Choose your new MacBook Pro . Students and educators — save on a new Mac for college. Get special pricing in the Education Store., title: Buy MacBook Pro - Apple, link: https://www.apple.com/shop/buy-mac/macbook-pro, snippet: 2024 MacBook Pro Laptop with M4 Pro , 14‑core CPU, 20‑core GPU: Built for Apple Intelligence, 16.2-inch Liquid Retina XDR Display, 48GB Unified Memory, 512GB SSD Storage; Space Black., title: Amazon.com: Macbook Pro, link: https://www.amazon.com/macbook-pro/s?k=macbook+pro, snippet: ** Price refers to MacBook Pro 14” with M4 or MacBook Air 13” with M2. Monthly pricin

{'input': 'What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?',
 'output': 'The current price of a MacBook Pro in USD is $1081, which would cost approximately €918.85 in EUR at an exchange rate of 0.85 EUR for 1 USD.'}

### ReAct Demo Code 2


In [ ]:
%%capture
# update or install the necessary libraries
!pip install --upgrade langchain
!pip install --upgrade python-dotenv
!pip install google-search-results


# import libraries
import openai
import os
from langchain.llms import OpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from dotenv import load_dotenv
load_dotenv()

# load API keys; you will need to obtain these if you haven't yet
os.environ["OPENAI_API_KEY"] = ""
os.environ["SERPER_API_KEY"] = ""

In [ ]:
!pip install openai==0.28

from langchain.chat_models import ChatOpenAI


llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
tools = load_tools(["google-serper", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

/tmp/ipython-input-54340317.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
/tmp/ipython-input-54340317.py:8: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/

In [ ]:
# run the agent
agent.run("Who are Mark Zukerberg's co-founders? What is their current age raised to the 1 power?")

/tmp/ipython-input-2577527173.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("Who are Mark Zukerberg's co-founders? What is their current age raised to the 1 power?")




> Entering new AgentExecutor chain...
I need to find out who Mark Zuckerberg's co-founders are and then calculate their current age raised to the power of 1.
Action: google_serper
Action Input: "Mark Zuckerberg co-founders"
Observation: Zuckerberg briefly attended Harvard College, and launched Facebook there in February 2004 with his roommates Eduardo Saverin, Andrew McCollum, Dustin Moskovitz and Chris Hughes . Zuckerberg took the company public in May 2012 with majority shares. Eduardo Luiz Saverin is a Brazilian billionaire entrepreneur, angel investor, and philanthropist, known for co-founding Facebook and later establishing the ... Where Mark Zuckerberg, Eduardo Saverin, Andrew McCollum, Dustin Moskovitz and Chris Hughes are today and what they are up to. Mark Zuckerberg (born May 14, 1984, White Plains, New York, U.S.) is an American computer programmer who was cofounder of the social networking site Facebook. Photo of Mark Zuckerberg, Dustin Moskovitz, Chris Hughes and Eduardo

'166'